   # Cycles and Trends

Let's explore the relationship between cycles and trends, with some short sequences.

We'll need routines to convert one to the other and vice-versa.

## Connecting the Dots

To explore the connections between permutation cycles and trendlists,
let's start by defining a function to help display our results.

In [1]:
def chain(elems, reverse=False):
    """Format a list as a causal chain,
    Default is left-to-right.
    reverse=True displays right-to-left (without reversing the causation.)
    """
    arrow = " -> "
    if reverse:
        arrow = " <- "
        elems = reversed(elems)
    return arrow.join([str(elem) for elem in elems]) 

print("Forwards: ", chain([1, 2, 3]))  # forwards
print("Backwards: ", chain([1, 2, 3], reverse=True)) # backwards

Forwards:  1 -> 2 -> 3
Backwards:  3 <- 2 <- 1


We'll also collect the imports we'll need in one place.

In [2]:
import itertools, math
from sympy.combinatorics import Permutation
from trendlist.simple import is_trend, pows, trend_list

## Cycles to Permutations and Back

We want to break permutations of an arbitrary finite sequence into its cycles.
We'll display the cycles as elements of $S_n$.

For example, if the original sequence is [1, 11, 121], then the permutation [1, 121, 11],
corresponding to indices [0, 2, 1], decomposes to the cycles [[0], [1, 2]].

We can display this as `[1, 121, 11] -> [0, 2, 1] -> [[0], [1, 2]]`

In [3]:
def seq_to_idxs(seq, orig):
    """Convert a sequence to indices.
    
    if seq is drawn from the original sequence, orig,
    return list of its indices from the original sequence
    """
    return [orig.index(elem) for elem in seq]

def idxs_to_cycles(idxs):
    """Return full cyclic decomposition of indices."""
    return Permutation(idxs).full_cyclic_form

def perm_idxs_cycles(perm, orig):
    "Return the triple (perm, idxs, cycles)"
    idxs = seq_to_idxs(perm, orig)
    cycles = idxs_to_cycles(idxs)
    return (perm, idxs, cycles)
    
def perm_to_cycles(perm, orig):
    "The entire transformation."
    _, _, cycles = perm_idxs_cycles(perm, orig)
    return cycles

And now the reverse.

In [4]:
def cycles_to_idxs(cycles):
    """Return the indices of a permutation.
    
    e.g., [[1], [0, 2]] -> [2, 1, 0]
    """
    return Permutation(cycles).array_form
    
def idxs_to_seq(idxs, orig):
    """Return the original sequence based on indices."""
    return [orig[idx] for idx in idxs]
    
def cycles_idxs_perm(cycles, orig):
    """Generate indices from cycles, then the permutation from the indices.
    Return the triple (cycles, indices, permutation)
    """
    idxs = cycles_to_idxs(cycles)
    perm = idxs_to_seq(idxs, orig)
    return (cycles, idxs, perm)

def cycles_to_perm(cycles, orig):
    """Turn cycles into the permutation, all in one go."""
    _, _, perm = cycles_idxs_perm(cycles, orig)
    return perm

A quick demo:

In [5]:
orig = [1, 11, 121, 1331]
cycles = [[0, 3], [1, 2]]
print("Go both directions.")
print(chain(cycles_idxs_perm(cycles, orig)))
perm = cycles_to_perm(cycles, orig)
print(chain(perm_idxs_cycles(perm, orig), reverse=True))

Go both directions.
[[0, 3], [1, 2]] -> [3, 2, 1, 0] -> [1331, 121, 11, 1]
[[0, 3], [1, 2]] <- [3, 2, 1, 0] <- [1331, 121, 11, 1]


## Trendlists to Cycles and Back

Now let's do a similar exercise for trendlists.

First we decompose a permutation of an arbitrary sequence into a trendlist, 
then turn the elements of the trendlist into their indices in the original sequence.

Because of the 1-1 correspondence between trendlists and permutation cycles, 
this corresponds to a cycle decomposition of the original sequence.

This maps the permutation to an element of $S_n$ through its trendlist.

In [6]:
def seq_to_trendlist(seq):
    """
    Map a sequence to its trendlist.
    """
    return trend_list(seq)
    
def trendlist_to_cycles(trendlist, orig):
    """Replace trendlist elements by indices from original sequence."""
    return [seq_to_idxs(trend, orig) for trend in trendlist]

def perm_trendlist_cycles(perm, orig):
    """Map permutation to a trendlist to cycles."""
    trendlist = seq_to_trendlist(perm)
    cycles = trendlist_to_cycles(trendlist, orig)
    return (perm, trendlist, cycles)

def perm_to_tcycles(perm, orig):
    """Permutation to "trendlist cycles" in one blow."""
    _, _, cycles = perm_trendlist_cycles(perm, orig)
    return cycles

In [7]:
orig = pows(4, base=11)
perm = [1, 1331, 11, 121]
print(chain(perm_trendlist_cycles(perm, orig)))

[1, 1331, 11, 121] -> [[1, 1331], [11, 121]] -> [[0, 3], [1, 2]]


And now the other direction.

In [8]:
def cycles_to_trendlist(cycles, orig):
    """Replace cycle indices by original values to turn cycles into a trendlist."""
    return[idxs_to_seq(cycle, orig) for cycle in cycles]

def trendlist_to_perm(trendlist):
    """Turn a trendlist into a permutation."""
    perm = []
    for trend in trendlist:
        perm.extend(trend)
    return perm

def cycles_trendlist_perm(cycles, orig):
    """Turn cycles into trendlists into permutations.
    
    Return triple (cycles, trendlist, perm)
    """
    trendlist = cycles_to_trendlist(cycles, orig)
    perm = trendlist_to_perm(trendlist)
    return (cycles, trendlist, perm)

def tcycles_to_perm(cycles, orig):
    _, _, perm = cycles_trendlist_perm(cycles, orig)
    return perm

orig = pows(4, base=11)
cycles = [[0, 3], [1, 2]]
print(chain(cycles_trendlist_perm(cycles, orig)))
perm = tcycles_to_perm(cycles, orig)
print(chain(perm_trendlist_cycles(perm, orig), reverse=True))

[[0, 3], [1, 2]] -> [[1, 1331], [11, 121]] -> [1, 1331, 11, 121]
[[0, 3], [1, 2]] <- [[1, 1331], [11, 121]] <- [1, 1331, 11, 121]


## Decompose All Permutations both Directions

Now we're ready to generate all permutations and decompose them both directions.

In [9]:
def all_permutations(seq):
    """Return all permutations of seq."""
    return [list(perm) for perm in itertools.permutations(seq)]

In [10]:
orig = pows(3)
for perm in all_permutations(orig):
    print(chain(perm_idxs_cycles(perm, orig), reverse=True))
    print(chain(perm_trendlist_cycles(perm, orig)))

[[0], [1], [2]] <- [0, 1, 2] <- [1, 2, 4]
[1, 2, 4] -> [[1, 2, 4]] -> [[0, 1, 2]]
[[0], [1, 2]] <- [0, 2, 1] <- [1, 4, 2]
[1, 4, 2] -> [[1, 4], [2]] -> [[0, 2], [1]]
[[0, 1], [2]] <- [1, 0, 2] <- [2, 1, 4]
[2, 1, 4] -> [[2, 1, 4]] -> [[1, 0, 2]]
[[0, 1, 2]] <- [1, 2, 0] <- [2, 4, 1]
[2, 4, 1] -> [[2, 4], [1]] -> [[1, 2], [0]]
[[0, 2, 1]] <- [2, 0, 1] <- [4, 1, 2]
[4, 1, 2] -> [[4], [1, 2]] -> [[2], [0, 1]]
[[0, 2], [1]] <- [2, 1, 0] <- [4, 2, 1]
[4, 2, 1] -> [[4], [2], [1]] -> [[2], [1], [0]]


Or, displayed more nicely,

In [11]:
orig = pows(4)
for perm in all_permutations(orig):
    cycles = perm_to_cycles(perm, orig)
    tcycles = Permutation(perm_to_tcycles(perm, orig)).full_cyclic_form
    print(f"{cycles} <- {perm} -> {tcycles}")

[[0], [1], [2], [3]] <- [1, 2, 4, 8] -> [[0, 1, 2, 3]]
[[0], [1], [2, 3]] <- [1, 2, 8, 4] -> [[0, 1, 3, 2]]
[[0], [1, 2], [3]] <- [1, 4, 2, 8] -> [[0, 2, 1, 3]]
[[0], [1, 2, 3]] <- [1, 4, 8, 2] -> [[0, 2, 3], [1]]
[[0], [1, 3, 2]] <- [1, 8, 2, 4] -> [[0, 3], [1, 2]]
[[0], [1, 3], [2]] <- [1, 8, 4, 2] -> [[0, 3], [1], [2]]
[[0, 1], [2], [3]] <- [2, 1, 4, 8] -> [[0, 2, 3, 1]]
[[0, 1], [2, 3]] <- [2, 1, 8, 4] -> [[0, 3, 2, 1]]
[[0, 1, 2], [3]] <- [2, 4, 1, 8] -> [[0, 3, 1, 2]]
[[0, 1, 2, 3]] <- [2, 4, 8, 1] -> [[0], [1, 2, 3]]
[[0, 1, 3, 2]] <- [2, 8, 1, 4] -> [[0, 2], [1, 3]]
[[0, 1, 3], [2]] <- [2, 8, 4, 1] -> [[0], [1, 3], [2]]
[[0, 2, 1], [3]] <- [4, 1, 2, 8] -> [[0, 1, 3], [2]]
[[0, 2, 3, 1]] <- [4, 1, 8, 2] -> [[0, 3, 2], [1]]
[[0, 2], [1], [3]] <- [4, 2, 1, 8] -> [[0, 3, 1], [2]]
[[0, 2, 3], [1]] <- [4, 2, 8, 1] -> [[0], [1, 3, 2]]
[[0, 2], [1, 3]] <- [4, 8, 1, 2] -> [[0, 1], [2, 3]]
[[0, 2, 1, 3]] <- [4, 8, 2, 1] -> [[0], [1], [2, 3]]
[[0, 3, 2, 1]] <- [8, 1, 2, 4] -> [[0, 1, 2], 

## Equivalence Classes

Because every sequence has exactly one circular permutation that is a single trend,
we can divide the $N!$ permutations into $(N-1)!$ equivalence classes.

What do they look like?

First collect all the single trends.

In [12]:
def single_trends(orig):
    """Return all permutations that are single trends."""
    return [perm for perm in all_permutations(orig) if len(perm_to_tcycles(perm, orig)) == 1]

single_trends([1, 2, 4, 8])

[[1, 2, 4, 8],
 [1, 2, 8, 4],
 [1, 4, 2, 8],
 [2, 1, 4, 8],
 [2, 1, 8, 4],
 [2, 4, 1, 8]]

We'll need something to collect all rotations of a sequence.

In [13]:
from collections import deque

In [14]:
def all_rotations(seq):
    """Return all rotations of a sequence."""
    de = deque(seq)
    rots = []
    for i in range(len(seq)):
        rots.append(list(de.copy()))
        de.rotate()
    return rots

all_rotations(pows(3))

[[1, 2, 4], [4, 1, 2], [2, 4, 1]]

And, finally, something that decomposes every permutation of an equivalence class.

In [20]:
orig = pows(4)

singles = single_trends(orig)
for trend in singles:
    print(f"== {trend} ==")
    for rot in all_rotations(trend):
        cycles = perm_to_cycles(rot, orig)
        tcycles = perm_to_tcycles(rot, orig)
        print(f"{cycles} <- {rot} -> {tcycles}")
    print()

== [1, 2, 4, 8] ==
[[0], [1], [2], [3]] <- [1, 2, 4, 8] -> [[0, 1, 2, 3]]
[[0, 3, 2, 1]] <- [8, 1, 2, 4] -> [[3], [0, 1, 2]]
[[0, 2], [1, 3]] <- [4, 8, 1, 2] -> [[2, 3], [0, 1]]
[[0, 1, 2, 3]] <- [2, 4, 8, 1] -> [[1, 2, 3], [0]]

== [1, 2, 8, 4] ==
[[0], [1], [2, 3]] <- [1, 2, 8, 4] -> [[0, 1, 3, 2]]
[[0, 2, 1], [3]] <- [4, 1, 2, 8] -> [[2], [0, 1, 3]]
[[0, 3, 1, 2]] <- [8, 4, 1, 2] -> [[3], [2], [0, 1]]
[[0, 1, 3], [2]] <- [2, 8, 4, 1] -> [[1, 3], [2], [0]]

== [1, 4, 2, 8] ==
[[0], [1, 2], [3]] <- [1, 4, 2, 8] -> [[0, 2, 1, 3]]
[[0, 3, 1], [2]] <- [8, 1, 4, 2] -> [[3], [0, 2], [1]]
[[0, 1, 3, 2]] <- [2, 8, 1, 4] -> [[1, 3], [0, 2]]
[[0, 2, 3], [1]] <- [4, 2, 8, 1] -> [[2, 1, 3], [0]]

== [2, 1, 4, 8] ==
[[0, 1], [2], [3]] <- [2, 1, 4, 8] -> [[1, 0, 2, 3]]
[[0, 3, 2], [1]] <- [8, 2, 1, 4] -> [[3], [1, 0, 2]]
[[0, 2, 1, 3]] <- [4, 8, 2, 1] -> [[2, 3], [1], [0]]
[[0], [1, 2, 3]] <- [1, 4, 8, 2] -> [[0, 2, 3], [1]]

== [2, 1, 8, 4] ==
[[0, 1], [2, 3]] <- [2, 1, 8, 4] -> [[1, 0, 3, 2]]
[[

## Observations

The cycles, on each side, make up $S_n$, broken into $(n-1)$ equivalence classes.
Because there's a 1-1 map, each has the same total cycle statistics: total number of cycles, distribution of cycle lengths, etc.

The cycles from direct cycle decomposition into full cyclic form, on the left, include one cyclic group, $C_n$, formed from the permutation $[0 ... n-1]$ and its rotations, $[n-1, 0, ... n-2]$, $[n-2, n-1, 0 ... n-3]$, and so on. The other equivalence groups are all cosets of this set.

The cycles from trend decomposition, on the right, also fall into corresponding equivalence classes. Each class has a lone, single trend, plus the cycles produced by trend decomposition of its $N$ rotations. No equivalence class is a group. The classes don't seem to be cosets of one another.

Each equivalence class on the left has exactly $N$ fixed points. Each equivalence class on the right has exactly one trend of length $N$.